In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Semantic Search using Embeddings

Semantic search is a type of search that uses the meaning of words and phrases to find relevant results.

In this tutorial, we will demonstrate how to do semantic search with embeddings generated from the news text and using [Google ScaNN: Efficient Vector Similarity Search](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) to retrieve the most relevant news semantically.

## Pre-requisites:
- Vertex LLM SDK
- ScaNN [github](https://github.com/google-research/google-research/tree/master/scann)

## Install Vertex LLM SDK

DISCLAIMER: Text Embedding API is now in Experimental Preview. This release focuses on validating model prototypes and these models are not guaranteed to be released. Use of Text Embedding API is governed by the Google Cloud Terms of Service, the Pre-GA Offerings Terms of the GCP Service Specific Terms. The Acceptance Use Policy, and the Generative AI Prohibited Use Policy. Vertex Text Embedding API’s features may be unstable, change in backward-incompatible ways, and are not guaranteed to be released. There are no SLAs provided and no technical support obligations. GCP’s Cloud Data Processing Addendum does not apply to Pre-GA Offerings and customers should not use Text Embedding API to process personal data or other data subject to legal or regulatory compliance requirements. See description of launch stage for details.

The information in this documentation is provided to the customer on an “as is” and “with all faults” basis without any warranty of any kind, either express or implied. Google does not warrant or guarantee the correctness, accuracy or reliability of the information in here. In no event will Google or its affiliates or licensors be liable for any damage or harm to customers from customer’s use of these materials.

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
PROJECT_ID = "[your-project-id]" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
!pip3 install google-cloud-aiplatform>=1.25

**Attention**: you would need to restart runtime so that the right package is installed.

## Import TextEmbeddingModel

In [ ]:
from vertexai.preview.language_models import TextEmbeddingModel
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

## Install ScaNN Package

In [ ]:
!pip install scann

## Imports packages

In [ ]:
import os
import time

import json
import numpy as np
import pandas as pd

import scann

## Create Embedding Dataset.

The dataset is solely to demonstrate the use of the Text Embedding API with a vector database. It is not intended to be used for any other purpose, such as evaluating models. The dataset is small and does not represent a comprehensive sample of all possible text.

In [ ]:
!gsutil cp gs://cloud-samples-data/vertex-ai/dataset-management/datasets/bert_finetuning/wide_and_deep_trainer_container_tests_input.jsonl .

In [ ]:
records = []
with open('wide_and_deep_trainer_container_tests_input.jsonl') as f:
  for line in f:
    record = json.loads(line)
    records.append(record)

In [ ]:
# Peek at the data.
df = pd.DataFrame(records)
df.head(50)

In [ ]:
def get_embedding(text):
  get_embedding.counter += 1
  try:
    if get_embedding.counter % 100 == 0:
      time.sleep(3)
    return model.get_embeddings([text])[0].values
  except:
    return []
get_embedding.counter = 0

# This may take several minutes to complete.
df['embedding'] = df['textContent'].apply(lambda x: get_embedding(x))

## Create an Index

In [ ]:
record_count = len(records)
dataset = np.empty((record_count, 768))
for i in range(record_count):
    dataset[i] = df.embedding[i]

normalized_dataset = dataset / np.linalg.norm(dataset, axis=1)[:, np.newaxis]
# configure ScaNN as a tree - asymmetric hash hybrid with reordering
# anisotropic quantization as described in the paper; see README

# use scann.scann_ops.build() to instead create a TensorFlow-compatible searcher
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product").tree(
    num_leaves=record_count, num_leaves_to_search=record_count, training_sample_size=record_count).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

## Queries the Index

In [ ]:
import time
def search(query):
    start = time.time()
    query = model.get_embeddings([query])[0].values
    neighbors, distances = searcher.search(query, final_num_neighbors=3)
    end = time.time()

    for id, dist in zip(neighbors, distances):
        print(f'[docid:{id}] [{dist}] -- {df.textContent[int(id)][:125]}...')
    print("Latency (ms):", 1000*(end - start))

In [ ]:
search("tell me about shark or animal")

In [ ]:
search("tell me about an important moment or event in your life")